### Libraries

In [54]:
from __future__ import annotations

import os
import math
import random
import numpy as np
from typing import Tuple
from dataclasses import dataclass

import torch
import torch.nn as nn
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn.functional as F

### Utilities

In [40]:
def grad_periodic(f: torch.Tensor, dx: float, dy: float) -> Tuple[torch.Tensor, torch.Tensor]:
  """Compute ∂f/∂x and ∂f/∂y with second-order central differences and periodic BCs."""
  f_xp = torch.roll(f, shifts=-1, dims=-1)
  f_xm = torch.roll(f, shifts=+1, dims=-1)
  dfdx = (f_xp - f_xm) / (2.0 * dx)

  f_yp = torch.roll(f, shifts=-1, dims=-2)
  f_ym = torch.roll(f, shifts=+1, dims=-2)
  dfdy = (f_yp - f_ym) / (2.0 * dy)

  return dfdx, dfdy

def laplacian_periodic(f: torch.Tensor, dx: float, dy: float) -> torch.Tensor:
  """5-point Laplacian with periodic BCs."""
  f_xp = torch.roll(f, shifts=-1, dims=-1)
  f_xm = torch.roll(f, shifts=+1, dims=-1)
  f_yp = torch.roll(f, shifts=-1, dims=-2)
  f_ym = torch.roll(f, shifts=+1, dims=-2)
  return (f_xp - 2.0 * f + f_xm) / (dx * dx) + (f_yp - 2.0 * f + f_ym) / (dy * dy)

### RK4 Integrator and Burger RHS

In [41]:
def burgers_rhs(u: torch.Tensor, v: torch.Tensor, nu: float, dx: float, dy: float) -> Tuple[torch.Tensor, torch.Tensor]:
  """Compute RHS of 2D vector Burgers for fields u, v on periodic grid."""
  u_x, u_y = grad_periodic(u, dx, dy)
  v_x, v_y = grad_periodic(v, dx, dy)

  adv_u = u * u_x + v * u_y
  adv_v = u * v_x + v * v_y

  lap_u = laplacian_periodic(u, dx, dy)
  lap_v = laplacian_periodic(v, dx, dy)

  du_dt = -adv_u + nu * lap_u
  dv_dt = -adv_v + nu * lap_v
  return du_dt, dv_dt

In [42]:
def rk4_step(u: torch.Tensor, v: torch.Tensor, dt: float, nu: float, dx: float, dy: float) -> Tuple[torch.Tensor, torch.Tensor]:
  k1_u, k1_v = burgers_rhs(u, v, nu, dx, dy)
  k2_u, k2_v = burgers_rhs(u + 0.5 * dt * k1_u, v + 0.5 * dt * k1_v, nu, dx, dy)
  k3_u, k3_v = burgers_rhs(u + 0.5 * dt * k2_u, v + 0.5 * dt * k2_v, nu, dx, dy)
  k4_u, k4_v = burgers_rhs(u + dt * k3_u, v + dt * k3_v, nu, dx, dy)

  u_next = u + (dt / 6.0) * (k1_u + 2 * k2_u + 2 * k3_u + k4_u)
  v_next = v + (dt / 6.0) * (k1_v + 2 * k2_v + 2 * k3_v + k4_v)

  return u_next, v_next

In [43]:
@torch.no_grad()
def generate_smooth_random_field(batch: int, H: int, W: int, strength: float = 0.5, cutoff: float = 8.0, device=None) -> torch.Tensor:
  """Generate smooth periodic random scalar fields via spectral filtering."""
  if device is None:
    device = torch.device("cpu")

  noise = torch.randn(batch, H, W, device=device, dtype=torch.float32)
  fhat = torch.fft.rfft2(noise)

  ky = torch.fft.fftfreq(H, d=1.0).to(device).view(H, 1)
  kx = torch.fft.rfftfreq(W, d=1.0).to(device).view(1, W // 2 + 1)
  k2 = ky ** 2 + kx ** 2

  filt = torch.exp(-k2 * (cutoff ** -2))
  fhat_filtered = fhat * filt

  field = torch.fft.irfft2(fhat_filtered, s=(H, W)).real

  # More conservative normalization
  field_std = field.std(dim=(-2, -1), keepdim=True)
  field = strength * field / torch.clamp(field_std, min=1e-6)

  # Clamp to prevent extreme value HELL
  field = torch.clamp(field, -2.0, 2.0)

  return field

In [45]:
@torch.no_grad()
def synthesize_dataset(N: int, H: int, W: int, T: float, steps: int, nu: float, device=None) -> Tuple[torch.Tensor, torch.Tensor]:
  """Create a dataset: inputs X=(u0,v0), targets Y=(uT,vT)."""
  if device is None:
    device = torch.device("cpu")

  Lx = 2 * math.pi
  Ly = 2 * math.pi

  dx = Lx / W
  dy = Ly / H
  dt = T / steps

  # Generate smoother initial conditions
  u0 = generate_smooth_random_field(N, H, W, strength=0.3, cutoff=10.0, device=device)
  v0 = generate_smooth_random_field(N, H, W, strength=0.3, cutoff=10.0, device=device)

  u = u0.clone()
  v = v0.clone()

  # Time integration with stability checks
  for step in range(steps):
    u, v = rk4_step(u, v, dt, nu, dx, dy)

    # Check for instabilities and clamp
    if torch.isnan(u).any() or torch.isnan(v).any():
      print(f"Warning: NaN detected at step {step}")
      break

    # To prevent extreme values during evolution
    u = torch.clamp(u, -5.0, 5.0)
    v = torch.clamp(v, -5.0, 5.0)

  X = torch.stack([u0, v0], dim=1)  # (N, 2, H, W)
  Y = torch.stack([u, v], dim=1)    # (N, 2, H, W)

  return X, Y

### FNO2D Layer and Model

In [46]:
class SpectralConv2d(nn.Module):
  """FIXED Fourier layer with proper initialization and stability."""

  def __init__(self, in_channels: int, out_channels: int, modes_x: int, modes_y: int):
    super().__init__()
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.modes_x = modes_x
    self.modes_y = modes_y

    # FIXED: Much smaller initialization for spectral weights
    scale = 1.0 / (in_channels * out_channels)
    init_std = scale / math.sqrt(modes_x * modes_y)

    # Complex weights as real parameters
    self.weight_real = nn.Parameter(torch.randn(in_channels, out_channels, modes_x, modes_y) * init_std)
    self.weight_imag = nn.Parameter(torch.randn(in_channels, out_channels, modes_x, modes_y) * init_std)

  def compl_mul2d(self, a, b_real, b_imag):
    """Complex multiplication with stability checks."""
    # a: (B, Cin, mx, my) - complex
    # b: (Cin, Cout, mx, my) - real and imag parts
    res_real = torch.einsum("bixy,ioxy->boxy", a.real, b_real) - torch.einsum("bixy,ioxy->boxy", a.imag, b_imag)
    res_imag = torch.einsum("bixy,ioxy->boxy", a.real, b_imag) + torch.einsum("bixy,ioxy->boxy", a.imag, b_real)

    # Stability: clamp extreme values
    res_real = torch.clamp(res_real, -1e6, 1e6)
    res_imag = torch.clamp(res_imag, -1e6, 1e6)

    return torch.complex(res_real, res_imag)

  def forward(self, x):
    B, C, H, W = x.shape

    # FIXED: Use standard normalization
    x_ft = torch.fft.rfft2(x)

    out_ft = torch.zeros(B, self.out_channels, H, W // 2 + 1,
                        device=x.device, dtype=torch.complex64)

    mx = min(self.modes_x, H // 2)
    my = min(self.modes_y, W // 2 + 1)

    # low-frequency block [0:mx, 0:my]
    out_ft[:, :, :mx, :my] = self.compl_mul2d(
      x_ft[:, :, :mx, :my],
      self.weight_real[:, :, :mx, :my],
      self.weight_imag[:, :, :mx, :my]
    )

    # high-frequency block [-mx:, 0:my] only if mx > 0
    if mx > 0 and mx < H // 2:
      out_ft[:, :, -mx:, :my] = self.compl_mul2d(
          x_ft[:, :, -mx:, :my],
          self.weight_real[:, :, :mx, :my],
          self.weight_imag[:, :, :mx, :my]
      )

    # FIXED: Standard inverse transform
    x_out = torch.fft.irfft2(out_ft, s=(H, W))

    return x_out

In [57]:
class FNO2D(nn.Module):
  def __init__(self, in_channels=2, width=32, modes_x=16, modes_y=16, layers=4, out_channels=2):
    super().__init__()
    self.width = width
    self.fc0 = nn.Conv2d(in_channels + 2, width, kernel_size=1) # +2 for positional encodings
    nn.init.kaiming_normal_(self.fc0.weight, nonlinearity='linear')
    nn.init.zeros_(self.fc0.bias)

    # Initialize SpectralConv2d with width as the input channels
    self.spectral_layers = nn.ModuleList([SpectralConv2d(width, width, modes_x, modes_y) for _ in range(layers)])
    self.w_layers = nn.ModuleList([nn.Conv2d(width, width, kernel_size=1) for _ in range(layers)])

    for w in self.w_layers:
      nn.init.kaiming_normal_(w.weight) # Removed nonlinearity='gelu'
      nn.init.zeros_(w.bias)

    self.act = nn.GELU()

    self.fc1 = nn.Conv2d(width, 64, kernel_size=1)
    self.fc2 = nn.Conv2d(64, out_channels, kernel_size=1)

    nn.init.kaiming_normal_(self.fc1.weight) # Removed nonlinearity='gelu'
    nn.init.zeros_(self.fc1.bias)

    nn.init.normal_(self.fc2.weight, std=1e-3)
    nn.init.zeros_(self.fc2.bias)

  def forward(self, x):
    # x: (B, 2, H, W)
    B, C, H, W = x.shape

    # positional encodings (normalized coords)
    gridx = torch.linspace(0, 1, W, device=x.device).view(1, 1, 1, W).repeat(B, 1, H, 1)
    gridy = torch.linspace(0, 1, H, device=x.device).view(1, 1, H, 1).repeat(B, 1, 1, W)

    x = torch.cat([x, gridx, gridy], dim=1)
    x = self.fc0(x)

    for spec, w in zip(self.spectral_layers, self.w_layers):
      res = x
      y1 = spec(x)
      y2 = w(x)
      x = self.act(y1 + y2 + res)


    x = self.act(self.fc1(x))
    x = self.fc2(x)

    return x

### Configuration

In [48]:
@dataclass
class Config:
  H: int = 64
  W: int = 64
  train_N: int = 256
  val_N: int = 32
  T: float = 0.1  # REDUCED time to prevent extreme values
  steps: int = 50  # REDUCED steps
  nu: float = 0.01
  batch_size: int = 8
  epochs: int = 20
  lr: float = 1e-4  # MUCH LOWER learning rate
  weight_decay: float = 1e-5
  modes_x: int = 8  # REDUCED modes
  modes_y: int = 8  # REDUCED modes
  width: int = 32
  layers: int = 3  # REDUCED layers
  out_dir: str = "artifacts"

### Dataset

In [49]:
class BurgersDataset(torch.utils.data.Dataset):
  def __init__(self, N, H, W, T, steps, nu, device):
    super().__init__()
    print(f"Generating {N} samples...")
    self.X, self.Y = synthesize_dataset(N, H, W, T, steps, nu, device=device)

    # Check for NaN/Inf in generated data
    if torch.isnan(self.X).any() or torch.isnan(self.Y).any():
      raise ValueError("NaN found in generated dataset!")
    if torch.isinf(self.X).any() or torch.isinf(self.Y).any():
      raise ValueError("Inf found in generated dataset!")

    # Conservative normalization
    self.X_mean = self.X.mean()
    self.X_std = self.X.std() + 1e-8
    self.Y_mean = self.Y.mean()
    self.Y_std = self.Y.std() + 1e-8

    self.X = (self.X - self.X_mean) / self.X_std
    self.Y = (self.Y - self.Y_mean) / self.Y_std

    print(f"Data stats - X: mean={self.X.mean():.4f}, std={self.X.std():.4f}")
    print(f"Data stats - Y: mean={self.Y.mean():.4f}, std={self.Y.std():.4f}")
    print(f"Data ranges - X: [{self.X.min():.4f}, {self.X.max():.4f}]")
    print(f"Data ranges - Y: [{self.Y.min():.4f}, {self.Y.max():.4f}]")

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]

### Train, Evalute and Plot

In [50]:
def train_one_epoch(model, loader, opt, scheduler=None):
  model.train()
  total_loss = 0.0
  num_batches = 0

  pbar = tqdm(loader, desc="Training", leave=False)
  for X, Y in pbar:
    X = X.to(device)
    Y = Y.to(device)

    if torch.isnan(X).any() or torch.isnan(Y).any():
      print("Warning: NaN in input data, skipping batch")
      continue

    opt.zero_grad()
    pred = model(X)

    if torch.isnan(pred).any():
        print("Warning: NaN in model output, skipping batch")
        continue

    loss = F.mse_loss(pred, Y)

    if torch.isnan(loss) or torch.isinf(loss):
      print("Warning: NaN/Inf loss detected, skipping batch")
      continue

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)

    opt.step()

    total_loss += loss.item()
    num_batches += 1

    pbar.set_postfix({'loss': f'{loss.item():.6f}'})

  if scheduler is not None:
    scheduler.step()

  return total_loss / max(num_batches, 1)

In [51]:
def evaluate(model, loader):
  model.eval()
  total = 0.0
  count = 0

  with torch.no_grad():
    for X, Y in loader:
      X = X.to(device)
      Y = Y.to(device)

      pred = model(X)

      if torch.isnan(pred).any():
        continue

      loss = F.mse_loss(pred, Y)

      if not torch.isnan(loss) and not torch.isinf(loss):
        total += loss.item() * X.size(0)
        count += X.size(0)

  return total / max(count, 1) if count > 0 else float('inf')

In [52]:
def quick_plot(sample_in, sample_true, sample_pred, out_path):
  """Plot u and v components side by side."""
  u0, v0 = sample_in[0], sample_in[1]
  uT_true, vT_true = sample_true[0], sample_true[1]
  uT_pred, vT_pred = sample_pred[0], sample_pred[1]

  fig, axs = plt.subplots(2, 3, figsize=(15, 8))

  # U component
  im0 = axs[0,0].imshow(u0, origin='lower', cmap='RdBu_r')
  axs[0,0].set_title('u₀ (Initial)')
  fig.colorbar(im0, ax=axs[0,0], fraction=0.046)

  im1 = axs[0,1].imshow(uT_true, origin='lower', cmap='RdBu_r')
  axs[0,1].set_title('u(T) True')
  fig.colorbar(im1, ax=axs[0,1], fraction=0.046)

  im2 = axs[0,2].imshow(uT_pred, origin='lower', cmap='RdBu_r')
  axs[0,2].set_title('u(T) Predicted')
  fig.colorbar(im2, ax=axs[0,2], fraction=0.046)

  # V component
  im3 = axs[1,0].imshow(v0, origin='lower', cmap='RdBu_r')
  axs[1,0].set_title('v₀ (Initial)')
  fig.colorbar(im3, ax=axs[1,0], fraction=0.046)

  im4 = axs[1,1].imshow(vT_true, origin='lower', cmap='RdBu_r')
  axs[1,1].set_title('v(T) True')
  fig.colorbar(im4, ax=axs[1,1], fraction=0.046)

  im5 = axs[1,2].imshow(vT_pred, origin='lower', cmap='RdBu_r')
  axs[1,2].set_title('v(T) Predicted')
  fig.colorbar(im5, ax=axs[1,2], fraction=0.046)

  for ax in axs.flat:
      ax.set_xticks([])
      ax.set_yticks([])

  plt.tight_layout()
  plt.savefig(out_path, dpi=150, bbox_inches='tight')
  plt.close(fig)

In [53]:
def plot_training_curves(train_losses, val_losses, out_path):
  """Plot training and validation loss curves."""
  fig, ax = plt.subplots(1, 1, figsize=(10, 6))

  epochs = range(1, len(train_losses) + 1)
  ax.plot(epochs, train_losses, 'b-', label='Training Loss', linewidth=2)
  ax.plot(epochs, val_losses, 'r-', label='Validation Loss', linewidth=2)

  ax.set_xlabel('Epoch')
  ax.set_ylabel('MSE Loss')
  ax.set_title('Training Progress')
  ax.legend()
  ax.grid(True, alpha=0.3)

  # Only use log scale if all losses are positive and finite
  valid_losses = [l for l in train_losses + val_losses if l > 0 and not math.isinf(l)]
  if valid_losses:
    ax.set_yscale('log')

  plt.tight_layout()
  plt.savefig(out_path, dpi=150, bbox_inches='tight')
  plt.close(fig)

### Main:
Make dataset -> Train -> Evaluation -> Plot

In [59]:
def main():
  # Set seeds for reproducibility
  SEED = 42
  random.seed(SEED)
  torch.manual_seed(SEED)
  np.random.seed(SEED)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  cfg = Config()
  os.makedirs(cfg.out_dir, exist_ok=True)

  print(f"Device: {device}")
  print("="*50)

  print("Generating datasets...")
  train_ds = BurgersDataset(cfg.train_N, cfg.H, cfg.W, cfg.T, cfg.steps, cfg.nu, device=device)
  val_ds = BurgersDataset(cfg.val_N, cfg.H, cfg.W, cfg.T, cfg.steps, cfg.nu, device=device)

  train_loader = torch.utils.data.DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True, drop_last=True)
  val_loader = torch.utils.data.DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False)

  print("\nBuilding model...")
  model = FNO2D(
      in_channels=2,
      width=cfg.width,
      modes_x=cfg.modes_x,
      modes_y=cfg.modes_y,
      layers=cfg.layers,
      out_channels=2
  ).to(device)

  total_params = sum(p.numel() for p in model.parameters())
  print(f"Total parameters: {total_params:,}")

  # Conservative optimizer settings
  opt = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
  scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=cfg.epochs)

  print(f"\nStarting training for {cfg.epochs} epochs with early stopping...")
  print("="*50)

  best_val = float('inf')
  train_losses = []
  val_losses = []
  patience = 10  # Number of epochs to wait for improvement
  epochs_no_improve = 0

  for epoch in range(1, cfg.epochs + 1):
    tr = train_one_epoch(model, train_loader, opt, scheduler)
    va = evaluate(model, val_loader)

    train_losses.append(tr)
    val_losses.append(va)

    print(f"Epoch {epoch:02d} | train MSE {tr:.6f} | val MSE {va:.6f} | lr {opt.param_groups[0]['lr']:.2e}")

    if va < best_val and not math.isnan(va) and not math.isinf(va):
        best_val = va
        torch.save(model.state_dict(), os.path.join(cfg.out_dir, "fno2d_burgers.pt"))
        print(f"  → New best model saved! (val loss: {va:.6f})")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve == patience:
        print(f"Early stopping triggered after {patience} epochs without improvement.")
        break

  # Plot training curves
  plot_training_curves(train_losses, val_losses, os.path.join(cfg.out_dir, "training_curves.png"))

  # Load best model for evaluation
  if os.path.exists(os.path.join(cfg.out_dir, "fno2d_burgers.pt")):
    model.load_state_dict(torch.load(os.path.join(cfg.out_dir, "fno2d_burgers.pt"), map_location=device))
    print(f"\nLoaded best model (val loss: {best_val:.6f})")
  else:
    print("No saved model found, using final model")
    torch.save(model.state_dict(), os.path.join(cfg.out_dir, "fno2d_burgers.pt"))

  # Qualitative evaluation
  model.eval()
  with torch.no_grad():
    X, Y = next(iter(val_loader))
    X = X.to(device)
    Y = Y.to(device)
    P = model(X)

  # Save comparison plot
  out_plot = os.path.join(cfg.out_dir, "qualitative_comparison.png")
  quick_plot(X[0].cpu().numpy(), Y[0].cpu().numpy(), P[0].cpu().numpy(), out_plot)

  print(f"\n" + "="*50)
  print(f"Training completed!")
  print(f"Best validation loss: {best_val:.6f}")
  print(f"Model saved to: {os.path.join(cfg.out_dir, 'fno2d_burgers.pt')}")
  print(f"Training curves: {os.path.join(cfg.out_dir, 'training_curves.png')}")
  print(f"Qualitative results: {out_plot}")

In [60]:
main()

Device: cpu
Generating datasets...
Generating 256 samples...
Data stats - X: mean=0.0000, std=1.0000
Data stats - Y: mean=-0.0000, std=1.0000
Data ranges - X: [-4.7879, 4.7834]
Data ranges - Y: [-6.4223, 5.8358]
Generating 32 samples...
Data stats - X: mean=-0.0000, std=1.0000
Data stats - Y: mean=-0.0000, std=1.0000
Data ranges - X: [-4.3759, 4.4995]
Data ranges - Y: [-4.9949, 5.0288]

Building model...
Total parameters: 398,786

Starting training for 20 epochs with early stopping...


Epoch 01 | train MSE 0.889266 | val MSE 0.767413 | lr 9.94e-05
  → New best model saved! (val loss: 0.767413)


Epoch 02 | train MSE 0.626617 | val MSE 0.463914 | lr 9.76e-05
  → New best model saved! (val loss: 0.463914)


Epoch 03 | train MSE 0.319986 | val MSE 0.187536 | lr 9.46e-05
  → New best model saved! (val loss: 0.187536)


Epoch 04 | train MSE 0.132630 | val MSE 0.101518 | lr 9.05e-05
  → New best model saved! (val loss: 0.101518)


...

Epoch 16 | train MSE 0.084241 | val MSE 0.084892 | lr 9.55e-06
  → New best model saved! (val loss: 0.084892)


Epoch 17 | train MSE 0.084214 | val MSE 0.084880 | lr 5.45e-06
  → New best model saved! (val loss: 0.084880)


Epoch 18 | train MSE 0.084199 | val MSE 0.084870 | lr 2.45e-06
  → New best model saved! (val loss: 0.084870)


Epoch 19 | train MSE 0.084189 | val MSE 0.084865 | lr 6.16e-07
  → New best model saved! (val loss: 0.084865)


Epoch 20 | train MSE 0.084184 | val MSE 0.084863 | lr 0.00e+00
  → New best model saved! (val loss: 0.084863)

Loaded best model (val loss: 0.084863)

Training completed!
Best validation loss: 0.084863
Model saved to: artifacts/fno2d_burgers.pt
Training curves: artifacts/training_curves.png
Qualitative results: artifacts/qualitative_comparison.png
